In [91]:
import pandas as pd
import sidrapy
import unicodedata
print(pd.__version__)

2.2.3


In [24]:
my_sheet = 'Sheet1'
df = pd.read_excel('./Plano_de_expansao_por_regiao.xlsx', sheet_name = my_sheet)
df

,Cidade,UF,Região,População,Pib
0,ABADIA DE GOIAS,GO,Centro-Oeste,NaN,NaN
1,ABADIA DOS DOURADOS,MG,Sudeste,NaN,NaN
2,ABADIANIA,GO,Centro-Oeste,NaN,NaN
3,ABAETE,MG,Sudeste,NaN,NaN
4,ABAIARA,CE,Nordeste,NaN,NaN
...,...,...,...,...,...
3374,XAXIM,SC,Sul,NaN,NaN
3375,XIQUE,XIQUE - BA,Desconhecido,NaN,NaN
3376,ZABELE,PB,Nordeste,NaN,NaN
3377,ZE DOCA,MA,Nordeste,NaN,NaN


In [26]:
data = sidrapy.get_table(table_code='9923',territorial_level="6",ibge_territorial_code="all", period="last 12")
data.head()

,NC,NN,MC,MN,V,D1C,D1N,D2C,D2N,D3C,D3N,D4C,D4N
0,Nível Territorial (Código),Nível Territorial,Unidade de Medida (Código),Unidade de Medida,Valor,Município (Código),Município,Ano (Código),Ano,Variável (Código),Variável,Situação do domicílio (Código),Situação do domicílio
1,6,Município,45,Pessoas,21494,1100015,Alta Floresta D'Oeste - RO,2022,2022,93,População residente,6795,Total
2,6,Município,45,Pessoas,96833,1100023,Ariquemes - RO,2022,2022,93,População residente,6795,Total
3,6,Município,45,Pessoas,5351,1100031,Cabixi - RO,2022,2022,93,População residente,6795,Total
4,6,Município,45,Pessoas,86887,1100049,Cacoal - RO,2022,2022,93,População residente,6795,Total


In [126]:
def limpar_texto(texto):
    # Remover acentos
    texto_sem_acento = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('ASCII')
    # Substituir espaços e hifens por vazio e converter para minúsculo
    texto_limpo = texto_sem_acento.replace(' ', '').replace('-', '').replace('(','').replace(')','').replace("'",'').lower()
    return texto_limpo

In [89]:
data_2 = data

In [92]:
data_2['D1N'] = data_2['D1N'].apply(limpar_texto)

In [99]:
data_2['D1N']

0                   municipio
1       altaflorestad'oestero
2                 ariquemesro
3                    cabixiro
4                    cacoalro
                ...          
5566             vianopolisgo
5567         vicentinopolisgo
5568                vilaboago
5569           vilapropiciogo
5570               brasiliadf
Name: D1N, Length: 5571, dtype: object

In [95]:
cidade_uf = df['Cidade'] + df['UF']

In [96]:
cidade_uf = cidade_uf.apply(limpar_texto)

In [98]:
cidade_uf

0           abadiadegoiasgo
1       abadiadosdouradosmg
2               abadianiago
3                  abaetemg
4                 abaiarace
               ...         
3374                xaximsc
3375           xiquexiqueba
3376               zabelepb
3377               zedocama
3378               zorteasc
Length: 3379, dtype: object

In [109]:
populacao_map = data_2.set_index('D1N')['V'].to_dict()

In [110]:
populacao_map
df2 = df

In [111]:
for i, nome_uf in enumerate(cidade_uf):
    if nome_uf in populacao_map:
        df2.at[i, 'População'] = int(populacao_map[nome_uf])
    else:
        df2.at[i, 'População'] = None  

In [112]:
df2

,Cidade,UF,Região,População,Pib
0,ABADIA DE GOIAS,GO,Centro-Oeste,19128.0,NaN
1,ABADIA DOS DOURADOS,MG,Sudeste,6272.0,NaN
2,ABADIANIA,GO,Centro-Oeste,17232.0,NaN
3,ABAETE,MG,Sudeste,22675.0,NaN
4,ABAIARA,CE,Nordeste,10038.0,NaN
...,...,...,...,...,...
3374,XAXIM,SC,Sul,31918.0,NaN
3375,XIQUE,XIQUE - BA,Desconhecido,44757.0,NaN
3376,ZABELE,PB,Nordeste,2228.0,NaN
3377,ZE DOCA,MA,Nordeste,40801.0,NaN


In [113]:
df2['População'].isnull().sum()

np.int64(24)

In [119]:
pib_data = pd.read_csv('./5938.csv',sep=';')
pib_data.head()

,Município,Pib
0,Alta Floresta D'Oeste (RO),734469
1,Ariquemes (RO),3209761
2,Cabixi (RO),238412
3,Cacoal (RO),2792383
4,Cerejeiras (RO),743037


In [127]:
pib_data2 = pib_data
pib_data2['Município'] = pib_data2['Município'].apply(limpar_texto)

In [128]:
pib_data2

,Município,Pib
0,altaflorestadoestero,734469
1,ariquemesro,3209761
2,cabixiro,238412
3,cacoalro,2792383
4,cerejeirasro,743037
...,...,...
5565,vianopolisgo,857968
5566,vicentinopolisgo,480173
5567,vilaboago,179899
5568,vilapropiciogo,308013


In [129]:
pib_map = pib_data2.set_index('Município')['Pib'].to_dict()

In [131]:
df3 = df2

In [134]:
for i, nome_uf in enumerate(cidade_uf):
    if nome_uf in populacao_map:
        df3.at[i, 'Pib'] = int(populacao_map[nome_uf])
    else:
        df3.at[i, 'Pib'] = None  

In [135]:
df3

,Cidade,UF,Região,População,Pib
0,ABADIA DE GOIAS,GO,Centro-Oeste,19128.0,19128.0
1,ABADIA DOS DOURADOS,MG,Sudeste,6272.0,6272.0
2,ABADIANIA,GO,Centro-Oeste,17232.0,17232.0
3,ABAETE,MG,Sudeste,22675.0,22675.0
4,ABAIARA,CE,Nordeste,10038.0,10038.0
...,...,...,...,...,...
3374,XAXIM,SC,Sul,31918.0,31918.0
3375,XIQUE,XIQUE - BA,Desconhecido,44757.0,44757.0
3376,ZABELE,PB,Nordeste,2228.0,2228.0
3377,ZE DOCA,MA,Nordeste,40801.0,40801.0


In [136]:
df3.to_csv('PopulacaoPib.csv')